In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error

In [76]:
df = pd.read_csv('./PJME_hourly.csv')

In [77]:
df.head()

,Datetime,PJME_MW
0,2002-12-31 01:00:00,26498.0
1,2002-12-31 02:00:00,25147.0
2,2002-12-31 03:00:00,24574.0
3,2002-12-31 04:00:00,24393.0
4,2002-12-31 05:00:00,24860.0


In [78]:
from sklearn.preprocessing import MinMaxScaler

In [79]:
scaler = MinMaxScaler()

In [80]:
import datetime

In [81]:
df.shape

(145366, 2)

In [82]:
df['Datetime'] = pd.to_datetime(df['Datetime'])

In [96]:
df_train = df.iloc[:116001]
df_train.shape

(116001, 2)

In [97]:
df_train.head()

,Datetime,PJME_MW
0,2002-12-31 01:00:00,26498.0
1,2002-12-31 02:00:00,25147.0
2,2002-12-31 03:00:00,24574.0
3,2002-12-31 04:00:00,24393.0
4,2002-12-31 05:00:00,24860.0


In [98]:
df_train.tail()

,Datetime,PJME_MW
115996,2015-10-06 05:00:00,21630.0
115997,2015-10-06 06:00:00,23570.0
115998,2015-10-06 07:00:00,26883.0
115999,2015-10-06 08:00:00,28450.0
116000,2015-10-06 09:00:00,28622.0


In [99]:
df_train.tail()

,Datetime,PJME_MW
115996,2015-10-06 05:00:00,21630.0
115997,2015-10-06 06:00:00,23570.0
115998,2015-10-06 07:00:00,26883.0
115999,2015-10-06 08:00:00,28450.0
116000,2015-10-06 09:00:00,28622.0


In [100]:
df_test = df.iloc[116001:]
df_test.shape

(29365, 2)

In [101]:
df_test.head()

,Datetime,PJME_MW
116001,2015-10-06 10:00:00,28588.0
116002,2015-10-06 11:00:00,28817.0
116003,2015-10-06 12:00:00,28838.0
116004,2015-10-06 13:00:00,28962.0
116005,2015-10-06 14:00:00,29169.0


In [106]:
scaler.fit(np.array(df_train["PJME_MW"]).reshape(-1, 1))
scaled_train = scaler.transform(np.array(df_train["PJME_MW"]).reshape(-1, 1))
scaled_test = scaler.transform(np.array(df_train["PJME_MW"]).reshape(-1, 1))

In [110]:
import pandas as pd
import numpy as np

def create_lstm_dataset(column, sequence_length=5):
    # data = []
    df_lstm = pd.DataFrame(columns=['t-5', 't-4', 't-3', 't-2', 't-1', 't'])
    
    # target_column = df.columns[0]  # Assuming the target column is the first column
    for index in range(len(column)):
        if(index <= 4):
            continue
        new_row = {'t-5': column[index-5] , 't-4':column[index-4], 't-3': column[index-3], 't-2': column[index-2], 't-1': column[index-1], 't': column[index]}
        df_row = pd.DataFrame(new_row, index=[0])
        df_lstm = pd.concat([df_lstm, df_row], ignore_index=True)
    
    return df_lstm


# # Example usage:
# # Assuming 'your_dataframe' is your hourly power consumption dataframe
X_y_train = create_lstm_dataset(scaled_train, sequence_length=5)
X_y_test = create_lstm_dataset(scaled_test, sequence_length=5)

In [125]:
X_train = [[[X_y_train["t-5"][index]], [X_y_train["t-4"][index]], [X_y_train["t-3"][index]], [X_y_train["t-2"][index]], [X_y_train["t-1"][index]]] for index in X_y_train.index]
y_train = X_y_train["t"]
X_test = [[[X_y_test["t-5"][index]], [X_y_test["t-4"][index]], [X_y_test["t-3"][index]], [X_y_test["t-2"][index]], [X_y_test["t-1"][index]]] for index in X_y_test.index]
y_test = X_y_test["t"]

In [127]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [120]:
import tensorflow as tf

In [133]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

In [134]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(5,1)))
model.add((tf.keras.layers.LSTM(64, return_sequences=True)))
model.add(tf.keras.layers.Dropout(0.2))
model.add((tf.keras.layers.LSTM(16, return_sequences=False)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 5, 64)             16896     
                                                                 
 dropout_2 (Dropout)         (None, 5, 64)             0         
                                                                 
 lstm_3 (LSTM)               (None, 16)                5184      
                                                                 
 dropout_3 (Dropout)         (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 22,097
Trainable params: 22,097
Non-trainable params: 0
_________________________________________________________________


In [135]:
cp = ModelCheckpoint('model1/', save_best_only=True)
model.compile(loss=MeanSquaredError(),
              optimizer=Adam(learning_rate=0.0001),
              metrics=[RootMeanSquaredError()]
              )

In [136]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, callbacks=[cp])

 954/3625 [======>.......................] - ETA: 14s - loss: 0.0210 - root_mean_squared_error: 0.1450

KeyboardInterrupt: 